In [1]:
import os
os.environ["JAVA_HOME"] = "D:\jdk"
os.environ["SPARK_HOME"] = "D:\spark"

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [4]:
df=spark.read.format('csv').option('header','true').load('original.csv')
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [9]:
#new column with clean data
from pyspark.sql.functions import *
df2=df.withColumn('JobTitle_clean', when (df.JobTitle.isNull(),'UNKOWN').otherwise(df.JobTitle))
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|      JobTitle_clean|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184| Assistant Professor|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|       Programmer II|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|              UNKOWN|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|Budget/Accounting...|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|            VP

In [10]:
#not null values
df2=df2.filter(df2.City.isNotNull())
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|      JobTitle_clean|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184| Assistant Professor|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|       Programmer II|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|Budget/Accounting...|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|            VP Sales|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|      Civil En

In [12]:
df2=df.withColumn('salary_int', df.Salary.substr(2,100).cast('int'))
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|salary_int|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|     57438|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|     57576|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|     61489|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|     63863|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$

In [13]:
average=df2.groupBy().avg('salary_int')
average.show()

+---------------+
|avg(salary_int)|
+---------------+
|      55487.457|
+---------------+



In [14]:
mean=df2.groupBy().avg('salary_int').take(1)[0][0]
print(mean)

55487.457
